In [1]:
import os
import random
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer, IBMQ
from qiskit.tools.visualization import plot_histogram
from dotenv import load_dotenv
# Load the API key from the .env file
load_dotenv()
IBM_TOKEN = os.getenv('IBM_TOKEN')
IBMQ.delete_account()
IBMQ.save_account(IBM_TOKEN)
IBMQ.load_account()

# Define functions to simplify interactions with quantum computers
def get_quantum_computer(qubits=2, simulation=False, verbose=False):
    # Finds a quantum computer at IBM with the lowest queue as long as it has enough qubits for the circuit
    if simulation == False: # Execute the circuit on a quantum computer at IBM
        lowest = float('inf')
        provider = IBMQ.get_provider(hub='ibm-q')
        for backend in provider.backends():
            try:
                if "simulator" not in backend.name():
                    queue = backend.status().pending_jobs
                    qubit_count = len(backend.properties().qubits)
                    if qubit_count >= qubits:
                        if queue < lowest:
                            lowest = queue
                            quantum_computer = provider.get_backend(backend.name())
                        if verbose == True:
                            print(f"Quantum Computer {backend.name()} has {queue} queued jobs and {qubit_count} qubits")
                    else:
                        if verbose == True:
                            print(f"Quantum Computer {backend.name()} has {qubit_count} qubits, but we need {qubits} qubits")
            except:
                print(f"Quantum Computer {backend.name()} is not operational")
        if quantum_computer is None:
            print(f"No Quantum Computers available with {qubits} qubits, using simulator")
    if simulation == True or quantum_computer is None: # Execute the circuit on the simulator
        quantum_computer = Aer.get_backend('qasm_simulator')
        if verbose == True:
            print(f"Using Quantum Computer Simulator")
    if verbose == True and simulation == False:
        print(f"Using Quantum Computer: {quantum_computer.name()} with {quantum_computer.status().pending_jobs} queued jobs")
    return quantum_computer

def prepare_quantum_circuit(qubits=2, classical_bits=2, simulation=False, verbose=False):
    # Create registers for the circuit
    quantum_register = QuantumRegister(qubits)
    classical_register = ClassicalRegister(classical_bits)
    # Create a Quantum Circuit
    quantum_circuit = QuantumCircuit(quantum_register, classical_register)
    # Get a quantum computer with at least the number of selected qubits for the circuit and the lowest queue, or the simulator
    quantum_computer = get_quantum_computer(qubits=qubits, simulation=simulation, verbose=verbose)
    return quantum_circuit, quantum_register, classical_register, quantum_computer

def execute_quantum_circuit(quantum_circuit, quantum_computer, shots=500, verbose=False):
    queue_position = quantum_computer.status().pending_jobs + 1
    if verbose == True:
        print(f"Your job is number {queue_position} in the queue on {quantum_computer.name()}.  Please wait...")
    result = execute(quantum_circuit, backend=quantum_computer, shots=shots).result()
    counts = result.get_counts(quantum_circuit)
    if verbose == True:
        print(f"{result.status} in {result.time_taken} seconds on {quantum_computer.name()}")
    highest_probable = result.get_counts(quantum_circuit).most_frequent()
    if verbose == True:
        probability = 100 * float(result.get_counts(quantum_circuit)[highest_probable])/float(shots)
        print(f"The probable result was {highest_probable} with {probability}% probability")
    return highest_probable, result, counts

def bell_state_circuit(quantum_circuit, quantum_register, classical_register):
    # Build a Bell State circuit
    # https://en.wikipedia.org/wiki/Bell_state
    quantum_circuit.h(quantum_register[0])
    quantum_circuit.cx(quantum_register[0], quantum_register[1])
    quantum_circuit.measure(quantum_register, classical_register)
    drawing = quantum_circuit.draw()
    return quantum_circuit, drawing

def brute_force_circuit(quantum_circuit, quantum_register, classical_register):
    # Build a brute force circuit using 3 qubits
    # https://en.wikipedia.org/wiki/Quantum_logic_gate#Quantum_logic_gates
    quantum_circuit.h(quantum_register[0])
    quantum_circuit.x(quantum_register[1])
    quantum_circuit.cx(quantum_register[0], quantum_register[1])
    quantum_circuit.h(quantum_register[2])
    quantum_circuit.cx(quantum_register[1], quantum_register[2])
    quantum_circuit.measure(quantum_register, classical_register)
    drawing = quantum_circuit.draw()
    return quantum_circuit, drawing

def iterate_words(words, shots=500, verbose=False):
    new_words = {}
    for word in words:
        quantum_circuit, quantum_register, classical_register, quantum_computer = prepare_quantum_circuit(qubits=3, classical_bits=3, simulation=True, verbose=True)
        quantum_circuit, drawing = brute_force_circuit(quantum_circuit, quantum_register, classical_register)
        highest_probable, result, counts = execute_quantum_circuit(quantum_circuit, quantum_computer, shots=shots, verbose=True)
        
        if highest_probable not in new_words:
            new_words[highest_probable] = word
        else:
            # I didn't like that answer, try again with less shots!
            if shots > 1:
                shots = int(shots/2)
            else:
                # We have exhausted our options, the gods have willed that this is not an option.
                continue
            new_words = iterate_words(words, quantum_circuit, quantum_register, classical_register, quantum_computer, shots=shots, verbose=verbose)
    return new_words


def brute_force(words, quantum_circuit, quantum_register, classical_register, quantum_computer, shots=500, verbose=False):
    # Lets do something silly.  Lets brute force a password if there are only 8 possible passwords.
    # We could wrap this in a function and pass in a dictionary then make this roll a probability for each, but we will just use a list for now.
    # There is no way this can be accurate, but it is fun to see the results.
    # Start by shuffling the list of words so that they're essentially in super position until observed below.
    random.shuffle(words)
    quantum_circuit, drawing = brute_force_circuit(quantum_circuit, quantum_register, classical_register)
    highest_probable, result, counts = execute_quantum_circuit(quantum_circuit, quantum_computer, shots=500, verbose=True)
    probabilities = {}
    for item in counts.keys():
        probability = 100 * float(counts[item])/float(500)
        for word in words:
            if word not in probabilities.keys():
                probabilities[word] = f"{probability}%"
    return probabilities, drawing, highest_probable, result, counts

In [5]:
# Set up the circuit
quantum_circuit, quantum_register, classical_register, quantum_computer = prepare_quantum_circuit(qubits=3, classical_bits=3, simulation=False, verbose=True)
# Run a brute force on the words list
words = ["password1", "password2", "password3", "password4", "password5", "password6", "password7", "password8"]
probabilities, drawing, highest_probable, result, counts = brute_force(words, quantum_circuit, quantum_register, classical_register, quantum_computer, shots=500, verbose=True)
drawing


Quantum Computer ibmq_lima has 31 queued jobs and 5 qubits
Quantum Computer ibmq_belem has 23 queued jobs and 5 qubits
Quantum Computer ibmq_quito has 8 queued jobs and 5 qubits
Quantum Computer ibmq_manila has 13 queued jobs and 5 qubits
Quantum Computer ibm_nairobi has 57 queued jobs and 7 qubits
Quantum Computer ibm_oslo has 15 queued jobs and 7 qubits
Using Quantum Computer: ibmq_quito with 7 queued jobs
Your job is number 8 in the queue on ibmq_quito.  Please wait...


In [ ]:
print(probabilities)

{'password8': '24.8%', 'password1': '24.8%', 'password6': '24.8%', 'password5': '24.8%', 'password3': '24.8%', 'password7': '24.8%', 'password4': '24.8%', 'password2': '24.8%'}
